In [13]:
from PIL import Image
import colorsys
import os
import math
import numpy as np

def hex_to_rgb(hex_color):
    # Convert a 6-char hexadecimal color to RGB tuple
    r = int(hex_color[0:2], 16)
    g = int(hex_color[2:4], 16)
    b = int(hex_color[4:6], 16)
    return r, g, b

def calculate_average_anomaly(image_path, top_left, bottom_right, map_for_rgb_to_anomaly):
    # Load the image
    img = Image.open(image_path)
    width, height = img.size

    # Extract the rectangle box region
    box_left = max(0, top_left[0])
    box_upper = max(0, top_left[1])
    box_right = min(width, bottom_right[0])
    box_lower = min(height, bottom_right[1])

    # Initialize the list to store the chart_anomaly_pixel_values
    chart_anomaly_pixel_values = []

    # Initialize counters for excluded and included pixels
    excluded_pixels = 0
    included_pixels = 0

    # Create map_for_rgb_hues_to_anomaly by converting hex RGB values to Hue values
    map_for_rgb_hues_to_anomaly = [colorsys.rgb_to_hsv(*hex_to_rgb(hex_color))[0] for hex_color in map_for_rgb_to_anomaly.keys()]

    # Iterate through the box and apply the mapping
    for y in range(box_upper, box_lower):
        for x in range(box_left, box_right):
            pixel_value = img.getpixel((x, y))
            
            # Check if the pixel meets the exclusion criteria (for both indexed PNG and RGB images)
            if isinstance(pixel_value, int):  # Indexed PNG image
                r, g, b = img.getpalette()[pixel_value * 3:pixel_value * 3 + 3]
            else:  # RGB image
                r, g, b = pixel_value

            # Check if the pixel meets the exclusion criteria
            h, s, v = colorsys.rgb_to_hsv(r / 255.0, g / 255.0, b / 255.0)
            if v * 100 < 38 or colorsys.rgb_to_hsv(r, g, b)[2] * 100 < 28:
                excluded_pixels += 1
                continue

            included_pixels += 1

            # Calculate the differences between the pixel Hue and map_for_rgb_hues_to_anomaly
            pixel_hue = colorsys.rgb_to_hsv(r / 255.0, g / 255.0, b / 255.0)[0]
            hue_differences = [abs(pixel_hue - hue) for hue in map_for_rgb_hues_to_anomaly]

            # Find the index of the mapping with the smallest difference
            min_diff_index = hue_differences.index(min(hue_differences))

            # Get the corresponding anomaly value from the map
            hex_color = list(map_for_rgb_to_anomaly.keys())[min_diff_index]
            chart_anomaly_pixel_values.append(map_for_rgb_to_anomaly[hex_color])

    # Calculate and print the average value
    if chart_anomaly_pixel_values:
        average_value = sum(chart_anomaly_pixel_values) / len(chart_anomaly_pixel_values)
        print(f"-- File Name: {image_path}, Average Value: {average_value}")

    # Print the number of excluded and included pixels
    print(f"--   Excluded Pixels: {excluded_pixels}, Included Pixels: {included_pixels}")
    return average_value

# Define the parameters for calculate_average_anomaly()
map_for_rgb_to_anomaly = {
    "ffcdba": -30,
    "f8bac5": -26,
    "f3a8d2": -22,
    "eb95dd": -19,
    "e684e8": -17,
    "df71f4": -15,
    "b65bec": -13,
    "9157db": -11,
    "6c52ca": -9,
    "484fb8": -7.5,
    "244ba6": -6.5,
    "004693": -5.5,
    "135ead": -4.5,
    "2874c6": -3.875,
    "3b8adf": -2.875,
    "64b7f8": -2.25,
    "78cdf7": -1.75,
    "8ce3f6": -1.25,
    "9ff8f4": -0.75,
    "ffffff": 0.0,
    "fffebe": 0.75,
    "feeaa0": 1.25,
    "fdd283": 1.75,
    "fdb365": 2.25,
    "f88b51": 2.875,
    "ef633e": 3.875,
    "dd3d2d": 4.5,
    "c21c26": 5.5,
    "9d2539": 6.5,
    "ba4355": 7.5,
    "d96073": 9,
    "f186a8": 11,
    "fda7dd": 13,
    "e497c0": 15,
    "cb86a4": 17,
    "b17386": 19,
    "9b646e": 22,
    "7f544f": 26,
    "66462f": 30
}

top_left = (35, 47)
bottom_right = (969, 491)

# Loop through all files in the "downloads/" folder
downloads_folder = "downloads/"

image_count = 0
day_average_values = []
for filename in sorted(os.listdir(downloads_folder)):
    
    if filename.lower().endswith(".png"):
        image_path = os.path.join(downloads_folder, filename)
        print(f"-- Processing {image_path}")
        period_average_value = calculate_average_anomaly(image_path, top_left, bottom_right, map_for_rgb_to_anomaly)
        # four periods in the day 00,06,12,18
        # use 0 index for image_count to figure out what period we are in
        day = math.floor(image_count / 4)
        if image_count % 4 == 0:
            day_average_values = []
        day_average_values.append(period_average_value)
        if image_count % 4 == 3:
            day_average = np.average(day_average_values)
            print(f"Day {day} average: {day_average}")
        image_count += 1        


-- Processing downloads/01.png
-- File Name: downloads/01.png, Average Value: 0.27622565936709126
--   Excluded Pixels: 32208, Included Pixels: 382488
-- Processing downloads/02.png
-- File Name: downloads/02.png, Average Value: 0.17743368613615576
--   Excluded Pixels: 32046, Included Pixels: 382650
-- Processing downloads/03.png
-- File Name: downloads/03.png, Average Value: 0.22353439478779633
--   Excluded Pixels: 32516, Included Pixels: 382180
-- Processing downloads/04.png
-- File Name: downloads/04.png, Average Value: 0.2850669278485145
--   Excluded Pixels: 31821, Included Pixels: 382875
Day 0 average: 0.24056516703488948
-- Processing downloads/05.png
-- File Name: downloads/05.png, Average Value: 0.3833534798198996
--   Excluded Pixels: 32909, Included Pixels: 381787
-- Processing downloads/06.png
-- File Name: downloads/06.png, Average Value: 0.32784692505675933
--   Excluded Pixels: 32380, Included Pixels: 382316
-- Processing downloads/07.png
-- File Name: downloads/07.png

In [ ]:
import os
import requests
import time
from bs4 import BeautifulSoup

def download_file(url, folder_path):
    response = requests.get(url)
    file_name = os.path.basename(url)
    file_path = os.path.join(folder_path, file_name)
    with open(file_path, 'wb') as f:
        f.write(response.content)

def get_file_urls(base_url):
    response = requests.get(base_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    urls = []
    for link in soup.find_all('a'):
        href = link.get('href')
        if href and not href.startswith('../') and href.startswith('geavg.t00z.pgrb2a.0p50_bcf') and 'idx' not in href:
            urls.append(href)
    return urls

# MAKE SURE THERE IS NO TRAILING SLASH
base_urls = [
    "https://nomads.ncep.noaa.gov/pub/data/nccf/com/naefs/prod/gefs.20230726/00/pgrb2ap5_bc",
    "https://nomads.ncep.noaa.gov/pub/data/nccf/com/naefs/prod/gefs.20230727/00/pgrb2ap5_bc"
]

num_urls = len(base_urls)
cur_url_count = 0
for base_url in base_urls:
    folder_path = os.path.basename(os.path.normpath(
        base_url.split('/')[-3]
    ))
    print(folder_path)
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    cur_url_count += 1
    print(f"Processing {cur_url_count} / {num_urls} folders to download GEFS, bias corrected, 2m ensemble mean temperatures")

    file_urls = get_file_urls(base_url)
    time.sleep(2)
    
    i = 0
    numfiles = len(file_urls)
    for file_url in file_urls:
        download_url = f"https://nomads.ncep.noaa.gov/cgi-bin/filter_gensbc.pl?dir=/{base_url.split('https://nomads.ncep.noaa.gov/pub/data/nccf/com/naefs/prod/')[-1]}&file={file_url}&var_TMP=on&lev_2_m_above_ground=on"
        i += 1
        print(f" - Downloading {i} / {numfiles} : {download_url}")
        download_file(download_url, folder_path)
        time.sleep(2)  # Pause for 2 seconds between downloads